<a href="https://colab.research.google.com/github/LoaoneSeleka/Data-Analysis-Projects/blob/main/Loaone_Seleka_Week_3_Project_Intermediate_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> 1. DUPLICATE THIS COLAB TO START WORKING ON IT. Using File > Save a copy to drive.
> 2. SHARE SETTINGS: In the new notebook, set the sharing settings to "Anyone with the link" by clicking "Share" on the top right corner.





Welcome to the third week's project for *Intermediate SQL*!

This week's lecture and material on CoRise showed you how to think about outer joins, set operators (union, intersect, except) and subqueries in SQL and combine that with other things you learned in Weeks 1 and 2. For this project we further your understanding of these concepts by delving a bit deeper. However, for everything covered in this project, you can find related examples in our course material.

# **Prerequisite configuration**
Below we install the software required to run this project. Please make sure to **RUN IT** by clicking on the play-button icon and to feel free to ignore the content of these two hidden cells.

In [2]:
%%capture
!pip install git+https://github.com/corise-edu/course-intermediate-sql.git

In [3]:
import pandas as pd
from IPython.display import display, HTML
from sql_course import run as sql_run
from sql_course import check

# Show all the rows (instead of only a few)
pd.set_option("display.max_rows", None)

# Set precision to max 2 decimals
pd.set_option('precision', 2)

# Set CSS Style for Table
# Make it work with night & light mode
# - Alternating rows
# - th elements
# - td elements
css_style = '''
<style>
  html {
    --td-font-color: black;
    --font-color: black;
    --background-color: #e0e0e0;
  }
  html[theme=dark] {
    --td-font-color: white;
    --font-color: black;
    --background-color: #6688ff;
  }
  th {
    background: #fbd44c;
    color: var(--font-color);
    font-size: 16px;
    text-align: center;
    font-weight: bold;
  }
  tr:nth-child(even) {
    background-color: var(--background-color);
    color: var(--font-color);
  }
  td {
    font-size: 14px;
    color: var(--td-font-color);
  }
</style>
'''


def run(sql_query):
  df = sql_run(sql_query)
  
  # Puts the scrollbar next to the DataFrame
  display(HTML(css_style + 
               "<div style='max-height: 500px; overflow: auto; width: fit-content; border-style: solid;" + 
               " border-width: 1px; border-color: #0139fe; font-family: GT Planar,Inter,Arial,sans-serif;'>" +
               df.style.render() +
               "</div>"))



# **The Setting:** 

Your life as a data analyst at CoRise has been a blast! Your previous successes helping CoRise understand the core drivers of their business have set you up very well at CoRise. You have been promoted to head analyst and are now expected to proactively ask the questions that will unearth value to the business folks at CoRise. 





# **Schema:**

As we discussed in Week 2 , the `courses` table from Week 1 has been replaced with the five tables below and we will continue to use these tables:


```
create table course_info (
course_id integer, 
course_name text,
course_desc text, 
course_category text, 
course_level text 
);
```

```
create table instructors (
instructor_id integer,
name text,
affiliation text,
teaching_experience integer
);
```

```
create table learners (
learner_id integer,
name text,
affiliation text
);
```

```
create table course_run (
course_run_id integer,
course_id integer,
instructor_id integer,
start_date date,
num_weeks integer,
num_TAs integer,
nps integer
);
```

```
create table course_registration_info (
course_run_id integer,
learner_id integer
);
```

# **Disclaimer**

Any resemblance of the data in this table to anything real (other than the names and descriptions of courses) are entirely random (could have been my pseudo random generator or my own imagination, since it isn't easy to distinguish between the two :-), so do not stress yourself out trying to make sense of any categorization, such as why a course run might even overlap with another run and such other niceties).

# **Part 0 - Wrapping Up Loose Ends from Week 2**

You may recall that we didn't do questions q_3_4 and q_4_2 from the Week 1 Project with the new set of normalized tables in Week 2. The reason we didn't attempt this in Week 2 is that it's a lot simpler to write these queries using subqueries. `num_learners_registered` was a column in the `courses` table during Week 1, but we added the concept of learners and their affiliations in the `learners` table. We also created yet another table that records which learner registered for which `course_run` in the `course_registration_info` table. As a result, `num_learners_registered` was not readily available. We will try our hand at q_4_2 again this week, but we'll leave q_3_4 for another day. (It turns out there is a bug in the `num_tas` column in the `course_run` table - it has negative values! This means the results from q_3_4 are not meaningful or intuitive, so we won't revisit it as part of this project.)

In [39]:
### Question: q3_0_1
### (Question q_4_2 from Week 1)
### Let's group the number of learners into three buckets:
### low (fewer than 30 learners), medium (30-100) and high (100+ learners).
### Let's get the minimum, maximum, & the average nps for each of these three buckets.
### Hint: You can compute num_learners_registered in a separate subquery and 
### provide it as a computed column to the outer query - after that it will look
### very similar to the situation where you had this as a column in the courses table.
### Output columns: num_learners_bucket, min_nps, max_nps, avg_nps
query = """
      
      SELECT
        cri.num_learners_bucket,
        MIN(cr.nps) AS min_nps,
        MAX(cr.nps) AS max_nps,
        AVG(cr.nps) AS avg_nps
      FROM course_run cr
      INNER JOIN  
              (SELECT course_run_id,(CASE 
                  WHEN COUNT(learner_id) < 30 THEN 'low'
                  WHEN COUNT(learner_id) >= 30 and COUNT(learner_id) < 100 THEN 'medium'
                  ELSE 'high'
            END) AS num_learners_bucket FROM course_registration_info GROUP BY course_run_id) cri
      WHERE cr.course_run_id = cri.course_run_id
      GROUP BY cri.num_learners_bucket

        """

run(query)
check(q3_0_1 = query)

,num_learners_bucket,min_nps,max_nps,avg_nps
0,high,41,99,73.92
1,low,56,100,83.55
2,medium,40,99,72.76


-------------------
Your SQL query is correct!


# **Part 1 - Digging into Outliers:**

As an expert analyst, by now you have recognized that understanding who the outliers are is usually very informative. Then you can explore reasons why these outliers exist and take action to remove the outliers if they are not desirable. 






In [36]:
### Question: q3_1_1
### Print the names of learners who have not registered for any course.
### Please use outer joins.
### You can assume course_run_id will never take null values
### in the table course_registration_info.
### Hint: In a left outer join, for a row in the left table with no matches
### in the right table what are the values in the columns of the right table?
### In particular if a learner has not registered for a course then
### what right hand side table can you left outer join the learners table (as the left table) with 
### to discover learners who have not registered for a course
### Output columns: name
query = """
        SELECT
          l.name
        FROM learners AS l
        LEFT JOIN course_registration_info AS cri
        ON cri.learner_id = l.learner_id
        GROUP BY cri.learner_id
        HAVING COUNT(cri.course_run_id) = 0;
                 
"""

run(query)
check(q3_1_1 = query)

,name
0,Javier Haddad


-------------------
Your SQL query is correct!


It may be a good idea to pass this information along to your chief of learner happiness, so they can get in touch and understand the reasons why this person (or these people) have not registered.

Before we move away from this query, let's try addressing the same question again, but this time try writing it without an outer join. This exercise is to illustrate how you can achieve the same results with very different SQL queries.

In [41]:
### Question: q3_1_2
### Print the names of learners who have not registered for any course.
### Please do not use outer joins. 
### Keep in mind that name isn't a primary key for the learners table (learner_id is).
### Appeal to the logic that learners who have not registered for any course
### are essentially a set of all learners minus the set of learners
### who have registered for some course. 
### Output columns: name

query = """
       SELECT
            name
       FROM learners
       WHERE learner_id NOT IN
            (SELECT
                learner_id
             FROM course_registration_info);
          
         """

run(query)
check(q3_1_2 = query)

,name
0,Javier Haddad


-------------------
Your SQL query is correct!


Well, that must have been satisfying, getting to write the same logic in two different ways.

On to a few more interesting outliers! CoRise is finding that it needs a bit more help running some longer courses (4 weeks or more in duration). It would be good to identify instructors who have never taught a course more than 3 weeks long, so the head of course content can connect with them and understand how to get them to teach longer courses (some have never taught a course at all!).

In [72]:
### Question: q3_1_3
### Retrieve ids and names of instructors who have never taught
### a course of more than 3 weeks long.
### Please use outer joins for this question.
### Output columns: instructor_id, name
query = """
  SELECT
      i.instructor_id,
      i.name
  FROM instructors AS i
  LEFT OUTER JOIN course_run AS cr
  ON
    i.instructor_id = cr.instructor_id AND cr.instructor_id IS NOT NULL
  GROUP BY i.instructor_id
  HAVING MAX(cr.num_weeks) <= 3 OR MAX(cr.num_weeks) IS NULL;
         """

run(query)
check(q3_1_3 = query)

,instructor_id,name
0,1,Damien Tapia
1,2,Paul Roth
2,3,Charlize Warner
3,4,Talon Foley
4,5,Maximo Duffy
5,6,Cornelius Hardin
6,7,Yahir Griffith
7,8,Elaina Brooks
8,9,Damarion Fernandez
9,10,Yosef Woods


-------------------
Your SQL query is correct!



Way to go! Now see if you can accomplish the same task without outer joins.

In [79]:
### Question: q3_1_4
### Retrieve ids and names of instructors who have never taught
### a course of more than 3 weeks long.
### Please do not use outer joins for this question.
### Output columns: instructor_id, name
query = """
    SELECT
      i.instructor_id,
      i.name 
    FROM instructors AS i
    WHERE i.instructor_id NOT IN
      (
        SELECT
          cr.instructor_id 
        FROM course_run AS cr
        WHERE cr.instructor_id IS NOT NULL
        GROUP BY cr.instructor_id
        HAVING MAX(cr.num_weeks) > 3
      )
        """

run(query)
check(q3_1_4 = query)

,instructor_id,name
0,1,Damien Tapia
1,2,Paul Roth
2,3,Charlize Warner
3,4,Talon Foley
4,5,Maximo Duffy
5,6,Cornelius Hardin
6,7,Yahir Griffith
7,8,Elaina Brooks
8,9,Damarion Fernandez
9,10,Yosef Woods


-------------------
Your SQL query is correct!


In [83]:
### Question: q3_1_5
### Print affiliations for which no learner
### has registered for a course in 2020.
### Please do not use outer joins for this query.
### Output columns: affiliation


query = """
  SELECT 
    lr.affiliation 
  FROM learners lr 
  WHERE lr.affiliation 
  NOT IN (SELECT DISTINCT l.affiliation 
          FROM course_registration_info cri
          INNER JOIN course_run cr 
            ON cr.course_run_id = cri.course_run_id
          INNER JOIN learners l 
            ON l.learner_id = cri.learner_id
          WHERE
          strftime("%Y", start_date) = '2020')
  GROUP BY
    lr.affiliation;
          

    
        """

run(query)
check(q3_1_5 = query)

,affiliation
0,Mail-Well Inc.


-------------------
Your SQL query is correct!


# **Part 2: Bonus Questions**

We will look at a few more complex queries to make these a bit more of a stretch for you.

In [4]:
### Question: q3_2_1
### Print ids of instructors whose max nps of any course they have facilitated 
### is smaller than the average nps of all the courses run.
###
### Hint:
### One possible approach is to create a subquery in the from clause.
### This subquery gets the average nps and renames it,
### so it can be used as a column in the outer query.
### This subquery returns one row and one column.
### It needs to be joined (using cross join) with other tables.

query = """
    SELECT 
      distinct r.instructor_id 
    FROM course_run cr
    INNER JOIN 
          (SELECT 
            instructor_id, max(nps) as max_nps 
          FROM course_run 
          WHERE instructor_id is not null 
          GROUP BY instructor_id
          ) r ON r.instructor_id = cr.instructor_id
          where cr.instructor_id is not null and r.max_nps < (select avg(nps) from course_run)
  
        """

run(query)
check(q3_2_1 = query)

,instructor_id
0,166
1,89
2,109
3,104
4,127
5,50
6,138
7,130
8,15


-------------------
Your SQL query is correct!


In [5]:
### Question: q3_2_2
### Retrieve affiliations of instructors who have collectively taught 
### all courses in the 'Deep Learning' category.
###
### Hint: 
### Note that this allows one instructor from affiliaiton X to teach one course
### and another instructor from the same affiliaiton X to teach another course 
### if there are two courses in the 'Deep Learning' category.
### Output columns: affiliation
query = """
        SELECT 
          i.affiliation from instructors i
        INNER JOIN course_run cr 
          ON cr.instructor_id = i.instructor_id
        INNER JOIN course_info ci 
          ON ci.course_id = cr.course_id
        WHERE 
          ci.course_category='Deep Learning' 
        GROUP BY 
          affiliation having count(distinct ci.course_id) = 
          (SELECT COUNT(course_id) 
          FROM course_info 
          WHERE course_category ='Deep Learning')  
        """

run(query)
check(q3_2_2 = query)

,affiliation
0,Host Marriott Corp.
1,Mellon Financial Corporation
2,Microsoft Corporation
3,NCR Corporation
4,Pioneer-Standard Electronics Inc.
5,Sequa Corp
6,Spartan Stores Inc
7,Tellabs Inc.
8,United Technologies Corporation


-------------------
Your SQL query is correct!


In [7]:
### Question: q3_2_3
### Print the sum of weeks spent on courses by learners and instructors per affiliation.
### Output columns: affiliation, sum_in_weeks


query = """
SELECT  
  r.affiliation, sum(cr3.num_weeks) as sum_in_weeks 
FROM 
    (
      SELECT i.affiliation,cr1.course_run_id 
      FROM instructors i
      INNER JOIN course_run cr1 
      ON cr1.instructor_id = i.instructor_id AND cr1.instructor_id is not null
      UNION ALL
      SELECT l.affiliation,cr2.course_run_id 
      FROM learners l
      INNER JOIN course_registration_info cri 
      ON cri.learner_id = l.learner_id
      INNER JOIN course_run cr2 on cr2.course_run_id = cri.course_run_id 
      WHERE l.learner_id is not null
                      )  r 
      INNER JOIN course_run cr3 on cr3.course_run_id = r.course_run_id
      GROUP BY  r.affiliation


        """

run(query)
check(q3_2_3 = query)

,affiliation,sum_in_weeks
0,3Com Corp,700
1,3M Company,626
2,A.G. Edwards Inc.,188
3,ABM Industries Incorporated,682
4,ACT Manufacturing Inc.,958
5,AFLAC Incorporated,877
6,AGCO Corporation,520
7,AK Steel Holding Corporation,722
8,ALLTEL Corporation,390
9,AMC Entertainment Inc.,225


-------------------
Your SQL query is correct!


# **Conclusion:** 

Wow! You are a SQL ninja - get ready to write some serious SQL queries and cause deep impact to your organization. Congratulations!